In [8]:

from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON

list_of_prefix="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX prop: <http://dbpedia.org/property/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX dbp: <http://dbpedia.org/resource/>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX res: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX db: <http://dbpedia.org/>
PREFIX yago: <http://dbpedia.org/class/yago/>
PREFIX onto: <http://dbpedia.org/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbc: <http://dbpedia.org/resource/Category:>
PREFIX dbpedia2: <http://dbpedia.org/property/>
"""

class Triple:
    def __init__(self, sub, pre, obj):
        self.sub = sub
        self.pre = pre
        self.obj = obj

    def get_subject(self):

        return self.sub

    def get_predicate(self):
        return self.pre

    def get_object(self):
        return self.obj


class Approx:
    def __init__(self, init_query,cost):
        self.query=SPARQL(init_query)
        self.cost=cost
        self.triples_to_query=[]
        self.original_query_t=[[], [], []]       
        self.approx_queries=dict()
        self.final_list = []
        self.xxx  = []
        self.get_triples()

        


    def get_triples(self):
        for s in self.query.abbr_triple_list:
            self.triples_to_query.append(
                Triple(s.split(' ')[0], s.split(' ')[1], s.split(' ')[2]))
            self.original_query_t[0].append(
                str(self.triples_to_query[-1].get_subject()))
            self.original_query_t[1].append(
                str(self.triples_to_query[-1].get_predicate()))
            self.original_query_t[2].append(
                str(self.triples_to_query[-1].get_object()))
        self.approximate()

    def approximate(self):
        for c in range(self.cost+1):
            value=[]
            if c == 0:
                value.append(self.triples_to_query)
                # build_query(value)
                self.execute_query(value, c)
            else:
                pre_approx=self.approx_queries[str(c-1)]
                # self.xxx.append(pre_approx)
                for triple in pre_approx:
                    new_approx=self.calculate(triple, c)
                    self.execute_query(new_approx, c)
                    # build_query(new_approx)
                    value += new_approx
            self.approx_queries[str(c)] = value
    
        

    def calculate(self, triples, cost):
        values=[]

        def left_insertion( index):
            list_of_approx=triples[:]
            list_of_approx.insert(index, (Triple(list_of_approx[index].get_subject(), f"?p{cost}", f"?o{cost}")))
            list_of_approx[index + 1]=Triple(f"?o{cost}", list_of_approx[index+1].get_predicate(), list_of_approx[index+1].get_object())
            return list_of_approx

        def right_insertion(index):
            list_of_approx=triples[:]
            list_of_approx.insert(index+1, (Triple(f"?o{cost}", f"?p{cost}", list_of_approx[index].get_object())))
            list_of_approx[index]=Triple(list_of_approx[index].get_subject(), list_of_approx[index].get_predicate(), f"?o{cost}")
            return list_of_approx

        def sub(index):
            list_of_approx=triples[:]
            list_of_approx[index]=(Triple(triples[index].get_subject(), f"?p{cost}", triples[index].get_object()))
            return list_of_approx
    


        for triple in triples:
            
            index=triples.index(triple)
            if (triple.get_subject() in self.original_query_t[0]):
                values.append(left_insertion(index))
            if (triple.get_predicate() in self.original_query_t[1]):
                values.append(sub(index))
            if (triple.get_object() in self.original_query_t[2]):
                values.append(right_insertion(index))
        return values

    

    def execute_query(self, queries, cost):

        def wr_my_list(l1list,cost):  
                file_path = 'test2.txt'
                with open(file_path, 'a') as file:
                    for l1 in zip(*l1list):
                        for l2 in l1:
                            self.final_list.append(l2)
                            file.write(str(l2) + " " + str(cost) + "\n" )

        def build_query(queries):
                sparql_query=f"{self.query.intent} distinct{self.query.former.strip(self.query.intent)} where {{\n"
                for triple in queries:
                    sparql_query += f"{triple.get_subject()} {triple.get_predicate()} {triple.get_object()} .\n"
                sparql_query += "}\n LIMIT 25"
                return (list_of_prefix + sparql_query)

        for query in queries:
            query_to_execute = build_query(query)
            sparql=SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
            sparql.setQuery(query_to_execute)
            sparql.setReturnFormat(JSON)
            results=sparql.query().convert()

            index=results['head']['vars']

            ind_reslut=[]
            result_list=[]


            for y in index:
                for x1 in results['results']['bindings']:
                    ind_reslut.append(x1[y]['value'])
                result_list.append(ind_reslut)
                ind_reslut=[]

            wr_my_list(result_list, cost)


# a = Approx(2)

## 1

In [18]:
def r1(triple):
    query = f"""
        SELECT ?a WHERE {{
            ?a rdfs:subPropertyOf {triple.get_object()}.
        }}
    """
    a = endpoint(list_of_prefix + query)
    if a:
        return (Triple(a, triple.get_predicate(),triple.get_object()))

## 2

In [14]:

from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON


def r2(triple):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?b WHERE {{
            {triple.get_predicate()} rdfs:subPropertyOf ?b.
        }}
    """
    b = endpoint(list_of_prefix + query)
    if b:
        return (Triple(triple.get_subject(), b,triple.get_object()))


## 3


In [17]:
def r3(triple):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?b WHERE {{
            ?a rdfs:subClassOf {triple.get_subject()}.
        }}
    """
    a = endpoint(list_of_prefix + query)
    if a:
        return (Triple(a,triple.get_predicate(),triple.get_object()))

## 4

In [25]:

from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON


def r4(triple):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?b WHERE {{
            {triple.get_object()} rdfs:subClassOf ?b.
        }}
    """
    b = endpoint(list_of_prefix + query)
    if b:
        return (Triple(triple.get_subject(), triple.get_predicate(),b))


## 5

In [32]:

from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON


def r5(triple):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?c WHERE {{
            {triple.get_predicate()} rdfs:domain ?c.
        }}
    """
    c = endpoint(list_of_prefix + query)
    if  c:
        return (Triple(triple.get_subject(),"<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>",c))


## 6

In [34]:

from typing import Any
from SPARQL_parser import SPARQL
from SPARQLWrapper import SPARQLWrapper, JSON


def r6(triple):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        SELECT ?d WHERE {{
            {triple.get_predicate()} rdfs:range ?d.
        }}
    """
    d = endpoint(list_of_prefix + query)
    if  d:
        return (Triple(triple.get_object(),"<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>",d))


In [66]:

triple = Triple("<http://dbpedia.org/resource/Albert_Einstein>", "<http://dbpedia.org/ontology/birthPlace>", "?place")
query = f"""
SELECT ?place
WHERE {{
   dbr:Albert_Einstein dbo:birthPlace ?place .
    
}}
"""

x =  relax(query,1)
if x:
    for i in x.final_list:
        print(i.get_subject(), i.get_predicate(), i.get_object())
    # print( x.final_list[0].get_subject(), x.final_list[0].get_predicate(), x.final_list[0].get_object())
else:
    print("nope")

http://dbpedia.org/ontology/aircraftBomber dbo:birthPlace ?place
dbr:Albert_Einstein http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#hasLocation ?place
dbr:Albert_Einstein dbo:birthPlace ['http://dbpedia.org/ontology/Document', 'http://dbpedia.org/ontology/SocietalEvent', 'http://dbpedia.org/ontology/PeriodicalLiterature', 'http://www.w3.org/2002/07/owl#Thing', 'http://dbpedia.org/ontology/Artist', 'http://dbpedia.org/ontology/Region', 'http://dbpedia.org/ontology/Actor', 'http://www.w3.org/2002/07/owl#Thing', 'http://dbpedia.org/ontology/PopulatedPlace', 'http://dbpedia.org/ontology/MeanOfTransportation', 'http://dbpedia.org/ontology/Company', 'http://dbpedia.org/ontology/Infrastructure', 'http://dbpedia.org/ontology/MusicalWork', 'http://www.w3.org/2002/07/owl#Thing', 'http://dbpedia.org/ontology/Boxer', 'http://dbpedia.org/ontology/Person', 'http://dbpedia.org/ontology/SportsLeague', 'http://dbpedia.org/ontology/GridironFootballPlayer', 'http://dbpedia.org/ontology/SportsTeam'

In [65]:
class relax:
    def __init__(self, init_query,cost):
        self.query=SPARQL(init_query)
        self.cost=cost
        self.triples_to_query=[]
        self.original_query_t=[[], [], []]       
        self.approx_queries=dict()
        self.final_list = []
        self.xxx  = []
        self.get_triples()  

    


    def get_triples(self):
        for s in self.query.abbr_triple_list:
            self.triples_to_query.append(
                Triple(s.split(' ')[0], s.split(' ')[1], s.split(' ')[2]))
            self.original_query_t[0].append(
                str(self.triples_to_query[-1].get_subject()))
            self.original_query_t[1].append(
                str(self.triples_to_query[-1].get_predicate()))
            self.original_query_t[2].append(
                str(self.triples_to_query[-1].get_object()))
        self.relax()

    def relax(self):
        for triple in self.triples_to_query:
            if self.r1(triple):
                self.final_list.append(self.r1(triple))
            if self.r2(triple):
                self.final_list.append(self.r2(triple))
            if self.r3(triple):
                self.final_list.append(self.r3(triple))
            if self.r4(triple):
                self.final_list.append(self.r4(triple))
            if self.r5(triple):
                self.final_list.append(self.r5(triple))
            if self.r6(triple):
                self.final_list.append(self.r6(triple))
    
    def r1(self,triple):
        query = f"""
            SELECT ?a WHERE {{
                ?a rdfs:subPropertyOf {triple.get_object()}.
            }}
        """
        a = self.endpoint(list_of_prefix + query)
        if a:
            return (Triple(a, triple.get_predicate(),triple.get_object()))
        
    
    def r2(self,triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?b WHERE {{
                {triple.get_predicate()} rdfs:subPropertyOf ?b.
            }}
        """
        b = self.endpoint(list_of_prefix + query)
        return (Triple(triple.get_subject(), b,triple.get_object()))
    

    def r3(self,triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?b WHERE {{
                ?a rdfs:subClassOf {triple.get_subject()}.
            }}
        """
        a = self.endpoint(list_of_prefix + query)
        if a:
            return (Triple(a,triple.get_predicate(),triple.get_object()))


    def r4(self,triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?b WHERE {{
                {triple.get_object()} rdfs:subClassOf ?b.
            }}
        """
        b = self.endpoint(list_of_prefix + query)
        if b:
            return (Triple(triple.get_subject(), triple.get_predicate(),b))
        

    
    def r5(self,triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?c WHERE {{
                {triple.get_predicate()} rdfs:domain ?c.
            }}
        """
        c = self.endpoint(list_of_prefix + query)
        if  c:
            return (Triple(triple.get_subject(),"<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>",c))


    def r6(self,triple):
        query = f"""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            SELECT ?d WHERE {{
                {triple.get_predicate()} rdfs:range ?d.
            }}
        """
        d = self.endpoint(list_of_prefix + query)
        if  d:
            return (Triple(triple.get_object(),"<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>",d))



        

    def endpoint(self,query):
        sparql = SPARQLWrapper("https://api.triplydb.com/datasets/DBpedia-association/dbpedia/services/dbpedia/sparql")
        sparql.setQuery(query)
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()

        index = results['head']['vars']

        ind_reslut = []
        result_list = []

        for y in index:
            for x1 in results['results']['bindings']:
                ind_reslut.append(x1[y]['value'])
            result_list += (ind_reslut)
            ind_reslut = []
        return (result_list[0])

### To do 

    1- get the labels instead of the uri or have both 
        1.1 for this you first need to get the variables if they are many you need to loop at the end to get them all 
    
    2- Find a way even if a query crashes you keep going on 
        2.1 i think here we need threads 

    3- add the empty path 

    4- continue from where it stopped 
        4.1 basically continue the dictionary 
            4.1.1 problem is it depends on the previous part so we need some sort of cashing to not repeat the whole cost cause for higher costs it impossible. 

        